<a href="https://colab.research.google.com/github/bhagyapatel/Big-Data/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !apt-get install openjdk-8-jdk-headless -qq > /dev/null
 !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
 !tar xf spark-3.0.0-bin-hadoop3.2.tgz
 !pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ["path"] ="${SPARK_HOME}/bin"
#spark-submit = /content/spark-3.0.0-bin-hadoop3.2/bin/spark-submit

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Project").getOrCreate()
from pyspark.sql.functions import *

In [ ]:
url = "https://raw.githubusercontent.com/azar-s91/dataset/master/weatherHistory.csv"

In [ ]:
from pyspark import SparkFiles,SparkContext

In [ ]:
spark.sparkContext.addFile(url)  

In [ ]:
df = spark.read.option("header",True).csv("file://" +SparkFiles.get("weatherHistory.csv"))

In [ ]:
df.show()

+---------------+---------------+--------+-----------------+--------------------+-------------+
|       DateTime|Temperature (C)|Humidity|Wind Speed (km/h)|Pressure (millibars)|      Summary|
+---------------+---------------+--------+-----------------+--------------------+-------------+
|4/1/06 12:00 AM|    9.472222222|    0.89|          14.1197|             1015.13|Partly Cloudy|
| 4/1/06 1:00 AM|    9.355555556|    0.86|          14.2646|             1015.63|Partly Cloudy|
| 4/1/06 2:00 AM|    9.377777778|    0.89|           3.9284|             1015.94|Mostly Cloudy|
| 4/1/06 3:00 AM|    8.288888889|    0.83|          14.1036|                null|Partly Cloudy|
| 4/1/06 4:00 AM|    8.755555556|    0.83|          11.0446|             1016.51|Mostly Cloudy|
| 4/1/06 5:00 AM|    9.222222222|    0.85|          13.9587|             1016.66|Partly Cloudy|
| 4/1/06 6:00 AM|    7.733333333|    0.95|          12.3648|             1016.72|Partly Cloudy|
| 4/1/06 7:00 AM|    8.772222222|    0.8

In [ ]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
df.withColumn("new_Date1", to_date("DateTime","MM/dd/yy")).show()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df = df.withColumnRenamed("Temperature (C)", "temp").withColumnRenamed("Wind Speed (km/h)","wind_speed") \
.withColumnRenamed("Pressure (millibars)","pressure")

In [ ]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+--------+----+--------+----------+--------+-------+
|DateTime|temp|Humidity|wind_speed|pressure|Summary|
+--------+----+--------+----------+--------+-------+
|       0|  11|       8|         9|       7|      0|
+--------+----+--------+----------+--------+-------+



In [ ]:
df.write.partitionBy("Summary").mode("overWrite").option("maxRecordPerPartition",2).csv("/content/sample_data/spark_df")

In [ ]:
df.write.partitionBy("Summary").mode("overwrite").format("csv").option("maxRecordPerPartition",2).saveAsTable("hive_df")

In [ ]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [ ]:
spark.sql("show partitions hive_df").show()

In [ ]:
spark.sql("show tables").show()

In [ ]:
spark.sql("select * from hive_df").show()

In [ ]:
output_valid = df.filter("'DateTime' is not null and \
 'Temperature (C)' is not null and \
 'Humidity' is not null and \
  'Wind Speed (km/h)' is not null and \
  'Pressure (millibars)' is not null")

In [ ]:
df_invalid = df.filter("'DateTime' is null or \
 'Temperature (C)' is null or \
 'Humidity' is null or \
  'Wind Speed (km/h)' is null or \
  'Pressure (millibars)' is null")

In [ ]:
df_invalid.write.option("header",True).mode("overwrite").saveAsTable("invalid_df")

In [ ]:
output_valid.write.option("header",True).mode("overwrite").saveAsTable("valid_df")

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [ ]:
dff = df.withColumn("date1", to_date(
    from_unixtime(
        unix_timestamp("DateTime", "MM/dd/yy hh:mm a")).cast(TimestampType()))).show()

+---------------+-----------+--------+----------+--------+-------------+----------+
|       DateTime|       temp|Humidity|wind_speed|pressure|      Summary|     date1|
+---------------+-----------+--------+----------+--------+-------------+----------+
|4/1/06 12:00 AM|9.472222222|    0.89|   14.1197| 1015.13|Partly Cloudy|2006-04-01|
| 4/1/06 1:00 AM|9.355555556|    0.86|   14.2646| 1015.63|Partly Cloudy|2006-04-01|
| 4/1/06 2:00 AM|9.377777778|    0.89|    3.9284| 1015.94|Mostly Cloudy|2006-04-01|
| 4/1/06 3:00 AM|8.288888889|    0.83|   14.1036|    null|Partly Cloudy|2006-04-01|
| 4/1/06 4:00 AM|8.755555556|    0.83|   11.0446| 1016.51|Mostly Cloudy|2006-04-01|
| 4/1/06 5:00 AM|9.222222222|    0.85|   13.9587| 1016.66|Partly Cloudy|2006-04-01|
| 4/1/06 6:00 AM|7.733333333|    0.95|   12.3648| 1016.72|Partly Cloudy|2006-04-01|
| 4/1/06 7:00 AM|8.772222222|    0.89|   14.1519| 1016.84|Partly Cloudy|2006-04-01|
| 4/1/06 8:00 AM|10.82222222|    0.82|   11.3183| 1017.37|Partly Cloudy|2006

In [ ]:
#df.withColumn("date1", to_date(current_timestamp())).show(1)
df1 = df.withColumn("DateOnly", 
                    to_date(
                        from_unixtime(
                            unix_timestamp("DateTime","MM/dd/yy hh:mm a")
                        ).cast(TimestampType())
                        )
                    )
                    

In [ ]:
df1.write.option("header",True)\
.mode("overwrite")\
.csv("df5")

In [ ]:
df1.withColumn("DateOnly", date_format("DateOnly","MM/dd/yyyy")).show()
df1.show()

In [ ]:
df1 = df1.withColumn("DateTime1",to_timestamp("DateTime","MM/dd/yy hh:mm"))
df1.show()

In [ ]:
#df1.withColumn("year",year("DateOnly")).show()
#df1.withColumn("month",month("DateOnly")).show()
#df1.withColumn("date_diff1",datediff(current_date(),"DateOnly")).show()
#df1.withColumn("day",dayofmonth("DateOnly")).show()
#df1.withColumn("hour1", hour("DateTime1")).show()
#df1.withColumn("week_of_year", weekofyear("DateTime1")).show() 
#df1.withColumn("dayinwords", date_format("DateOnly","EEE")).show()
#df1.withColumn("monthinwords", date_format("DateOnly","LLL")).show()
#df1.withColumn("new_Date", add_months("DateOnly",3)).show()
df1.withColumn("new_Date", date_add("DateOnly",3)).show()


+---------------+-----------+--------+----------+--------+-------------+----------+-------------------+----------+
|       DateTime|       temp|Humidity|wind_speed|pressure|      Summary|  DateOnly|          DateTime1|  new_Date|
+---------------+-----------+--------+----------+--------+-------------+----------+-------------------+----------+
|4/1/06 12:00 AM|9.472222222|    0.89|   14.1197| 1015.13|Partly Cloudy|2006-04-01|2006-04-01 00:00:00|2006-04-04|
| 4/1/06 1:00 AM|9.355555556|    0.86|   14.2646| 1015.63|Partly Cloudy|2006-04-01|2006-04-01 01:00:00|2006-04-04|
| 4/1/06 2:00 AM|9.377777778|    0.89|    3.9284| 1015.94|Mostly Cloudy|2006-04-01|2006-04-01 02:00:00|2006-04-04|
| 4/1/06 3:00 AM|8.288888889|    0.83|   14.1036|    null|Partly Cloudy|2006-04-01|2006-04-01 03:00:00|2006-04-04|
| 4/1/06 4:00 AM|8.755555556|    0.83|   11.0446| 1016.51|Mostly Cloudy|2006-04-01|2006-04-01 04:00:00|2006-04-04|
| 4/1/06 5:00 AM|9.222222222|    0.85|   13.9587| 1016.66|Partly Cloudy|2006-04-

In [ ]:
!ls

df5	     spark-3.0.0-bin-hadoop3.2	    spark-warehouse
sample_data  spark-3.0.0-bin-hadoop3.2.tgz


In [ ]:
!hadoop fs -cat /content/test2.txt

/bin/bash: hadoop: command not found


In [ ]:
#df.withColumn("date1", to_date("DateTime","MM/dd/yy hh:mm a")).show(1)

In [ ]:
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os
def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [ ]:
#@title Colab Shell
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });

In [ ]:
!ls

project.py   spark-3.0.0-bin-hadoop3.2	    untitled14.py
sample_data  spark-3.0.0-bin-hadoop3.2.tgz


In [ ]:
!spark-submit --master local untitled14.py

/bin/bash: spark-submit: command not found
